# 因果发现 - 从数据中学习因果结构

## 业务场景：电商平台用户流失预警

你是某电商平台的数据科学家。产品团队收集了用户的多个行为特征：
- 浏览时长 (BrowseTime)
- 购买频次 (PurchaseFreq)
- 客服投诉 (Complaints)
- 优惠券使用 (CouponUsage)
- 会员等级 (MemberLevel)
- 是否流失 (Churn)

产品经理问你：**"这些因素之间到底是什么因果关系？应该优先改善哪个环节？"**

传统回归只能告诉你相关性，而**因果发现**能帮你找到真正的因果结构！

---

## 学习目标

1. 理解因果发现与因果推断的区别
2. 掌握 PC 算法的原理和实现
3. 了解 LiNGAM、GES 等其他方法
4. 学会评估和解释因果图
5. 理解因果发现的局限性

In [ ]:
# 环境准备
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import networkx as nx
import warnings
warnings.filterwarnings('ignore')

# 因果发现库
try:
    from causallearn.search.ConstraintBased.PC import pc
    from causallearn.search.ScoreBased.GES import ges
    from causallearn.utils.GraphUtils import GraphUtils
    from causallearn.utils.cit import chisq, fisherz
except ImportError:
    print("正在安装 causal-learn...")
    import sys
    !{sys.executable} -m pip install causal-learn -q
    from causallearn.search.ConstraintBased.PC import pc
    from causallearn.search.ScoreBased.GES import ges
    from causallearn.utils.GraphUtils import GraphUtils
    from causallearn.utils.cit import chisq, fisherz

# 设置随机种子
np.random.seed(42)

# Plotly 配置
import plotly.io as pio
pio.templates.default = "plotly_white"

print("✅ 环境准备完成")

---

# Part 1: 因果发现的问题

## 1.1 从相关到因果

### 问题定义

**因果推断 (Causal Inference)**：
- 已知因果图 $G$
- 估计因果效应 $\tau = E[Y(1) - Y(0)]$

**因果发现 (Causal Discovery)**：
- 未知因果图
- 从数据 $D$ 中学习因果结构 $G$

### 核心挑战

**相关性不等于因果性**：
$$
Corr(X, Y) \neq 0 \text{ 可能因为:}
\begin{cases}
X \rightarrow Y & \text{(X 导致 Y)} \\
Y \rightarrow X & \text{(Y 导致 X)} \\
X \leftarrow Z \rightarrow Y & \text{(共同原因)} \\
X \rightarrow Z \leftarrow Y & \text{(共同结果)}
\end{cases}
$$

### 可识别性

**马尔可夫等价类 (Markov Equivalence Class)**：
- 相同条件独立性的因果图集合
- 只能学到 CPDAG (Completed Partially DAG)
- 某些边的方向无法确定

In [ ]:
# 演示：三种相关关系
def generate_correlation_scenarios():
    n = 1000
    
    # Scenario 1: X -> Y (因果)
    X1 = np.random.randn(n)
    Y1 = 2 * X1 + np.random.randn(n) * 0.5
    
    # Scenario 2: X <- Z -> Y (混淆)
    Z = np.random.randn(n)
    X2 = Z + np.random.randn(n) * 0.3
    Y2 = Z + np.random.randn(n) * 0.3
    
    # Scenario 3: X -> Z <- Y (对撞)
    X3 = np.random.randn(n)
    Y3 = np.random.randn(n)
    Z3 = X3 + Y3 + np.random.randn(n) * 0.3
    
    return [(X1, Y1, "X → Y (Causation)"),
            (X2, Y2, "X ← Z → Y (Confounding)"),
            (X3, Y3, "X → Z ← Y (Collider)")]

scenarios = generate_correlation_scenarios()

# 可视化
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=[s[2] for s in scenarios]
)

for i, (X, Y, title) in enumerate(scenarios, 1):
    corr = np.corrcoef(X, Y)[0, 1]
    
    fig.add_trace(
        go.Scatter(
            x=X, y=Y, mode='markers',
            marker=dict(size=4, color='#2D9CDB', opacity=0.5),
            name=f"Corr = {corr:.3f}",
            showlegend=False
        ),
        row=1, col=i
    )
    
    # 添加注释
    fig.add_annotation(
        text=f"Correlation: {corr:.3f}",
        xref=f"x{i}", yref=f"y{i}",
        x=X.mean(), y=Y.max(),
        showarrow=False,
        bgcolor="lightyellow"
    )

fig.update_layout(
    title_text="相关性的三种来源 - 为什么需要因果发现?",
    height=400,
    showlegend=False
)
fig.update_xaxes(title_text="X")
fig.update_yaxes(title_text="Y")
fig.show()

print("💡 关键洞察：")
print("三个场景都有显著相关性，但因果结构完全不同！")
print("只有因果发现才能区分它们。")

## 1.2 准备业务数据

让我们生成电商用户流失数据，其中隐含了真实的因果结构。

In [ ]:
def generate_ecommerce_data(n=2000):
    """
    真实因果结构:
    MemberLevel -> PurchaseFreq -> BrowseTime
                \                    ↓
                 -> CouponUsage -> Churn
                         ↑           ↑
                    Complaints ------┘
    """
    # 外生变量
    member_level = np.random.choice([1, 2, 3], n, p=[0.5, 0.3, 0.2])
    
    # 会员等级影响购买频次
    purchase_freq = 2 * member_level + np.random.randn(n) * 1.5
    purchase_freq = np.clip(purchase_freq, 0, 15)
    
    # 购买频次影响浏览时长
    browse_time = 10 + 5 * purchase_freq + np.random.randn(n) * 10
    browse_time = np.clip(browse_time, 0, 120)
    
    # 独立的投诉
    complaints = np.random.poisson(lam=0.5, size=n)
    
    # 优惠券使用受会员等级和投诉影响
    coupon_usage = 3 * member_level - 2 * complaints + np.random.randn(n) * 2
    coupon_usage = np.clip(coupon_usage, 0, 15)
    
    # 流失受浏览时长、优惠券、投诉影响
    logit = -2 - 0.05 * browse_time - 0.3 * coupon_usage + 0.8 * complaints
    churn_prob = 1 / (1 + np.exp(-logit))
    churn = (np.random.rand(n) < churn_prob).astype(int)
    
    df = pd.DataFrame({
        'MemberLevel': member_level,
        'PurchaseFreq': purchase_freq,
        'BrowseTime': browse_time,
        'Complaints': complaints,
        'CouponUsage': coupon_usage,
        'Churn': churn
    })
    
    return df

# 生成数据
df = generate_ecommerce_data(2000)

print("📊 数据概览:")
print(df.head())
print("\n基本统计:")
print(df.describe())
print(f"\n流失率: {df['Churn'].mean():.2%}")

In [ ]:
# 相关性矩阵 (这是我们能直接观察到的)
corr_matrix = df.corr()

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix.values,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu',
    zmid=0,
    text=corr_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 10},
    colorbar=dict(title="Correlation")
))

fig.update_layout(
    title="变量相关性矩阵 - 但因果关系是什么?",
    height=500,
    xaxis={'side': 'bottom'}
)
fig.show()

print("❓ 问题：")
print("从相关性矩阵，你能判断出因果方向吗？")
print("MemberLevel 和 PurchaseFreq 相关 0.67，但谁导致谁？")
print("\n这就是因果发现要解决的问题！")

---

# Part 2: 基于约束的方法 - PC 算法

## 2.1 核心思想

PC (Peter-Clark) 算法基于**条件独立性检验**学习因果结构。

### 理论基础

**d-分离 (d-separation)**：
- 如果 $X \perp\!\!\!\perp Y | Z$ (在 DAG $G$ 中)
- 则 $P(X, Y | Z) = P(X | Z) P(Y | Z)$ (在数据分布 $P$ 中)

**马尔可夫条件**：
$$
X_i \perp\!\!\!\perp \text{NonDesc}(X_i) | \text{Pa}(X_i)
$$

### 算法流程

**Step 1: 构建完全无向图**
- 所有变量两两相连

**Step 2: 删边 (基于条件独立性)**
- 对每条边 $X - Y$，寻找条件集 $S$
- 如果 $X \perp\!\!\!\perp Y | S$，删除该边

**Step 3: 定向边 (基于 v-结构)**
- 找到 $X - Z - Y$ 且 $X, Y$ 不相邻
- 如果 $Z \notin S_{XY}$，则 $X \rightarrow Z \leftarrow Y$ (对撞结构)

**Step 4: 传播方向**
- 避免产生新的 v-结构
- 避免产生环

In [ ]:
# 手动实现简化版 PC 算法（教学用）
from scipy.stats import pearsonr, chi2_contingency
from itertools import combinations

def independence_test(data, X, Y, Z=None, alpha=0.05):
    """
    条件独立性检验: X ⊥ Y | Z
    使用偏相关系数
    """
    if Z is None or len(Z) == 0:
        # 边缘独立性
        corr, pval = pearsonr(data[X], data[Y])
        return pval > alpha
    else:
        # 条件独立性 (简化：用偏相关)
        from sklearn.linear_model import LinearRegression
        
        # 残差化
        reg_X = LinearRegression().fit(data[Z], data[X])
        reg_Y = LinearRegression().fit(data[Z], data[Y])
        
        res_X = data[X] - reg_X.predict(data[Z])
        res_Y = data[Y] - reg_Y.predict(data[Z])
        
        corr, pval = pearsonr(res_X, res_Y)
        return pval > alpha

def simple_pc_skeleton(data, alpha=0.05, max_cond_size=2):
    """
    PC 算法 Step 1-2: 学习骨架
    """
    variables = list(data.columns)
    n_vars = len(variables)
    
    # 初始化完全图
    adjacency = {var: set(variables) - {var} for var in variables}
    separation_sets = {}
    
    # 逐步增加条件集大小
    for cond_size in range(max_cond_size + 1):
        for X in variables:
            for Y in list(adjacency[X]):
                if Y not in adjacency[X]:
                    continue
                
                # 候选条件集
                candidates = adjacency[X] - {Y}
                
                if len(candidates) < cond_size:
                    continue
                
                # 检验所有大小为 cond_size 的条件集
                for Z in combinations(candidates, cond_size):
                    Z_list = list(Z)
                    if independence_test(data, X, Y, Z_list, alpha):
                        # 找到分离集，删边
                        adjacency[X].discard(Y)
                        adjacency[Y].discard(X)
                        separation_sets[(X, Y)] = Z_list
                        separation_sets[(Y, X)] = Z_list
                        break
    
    return adjacency, separation_sets

# 运行简化版 PC
print("🔍 运行 PC 算法（简化版）...")
adjacency, sep_sets = simple_pc_skeleton(df, alpha=0.01, max_cond_size=2)

print("\n学到的邻接关系:")
for var, neighbors in adjacency.items():
    if neighbors:
        print(f"{var}: {neighbors}")

print("\n找到的分离集:")
for (X, Y), S in sep_sets.items():
    if X < Y:  # 只打印一次
        print(f"{X} ⊥ {Y} | {S}")

## 2.2 使用 causal-learn 库

实际应用中，我们使用成熟的库进行因果发现。

In [ ]:
# 准备数据
data_array = df.values
var_names = list(df.columns)

# 运行 PC 算法
print("🔍 运行 PC 算法 (causal-learn)...")
cg = pc(
    data_array,
    alpha=0.05,
    indep_test='fisherz',  # Fisher's Z 检验
    stable=True,
    uc_rule=0,
    uc_priority=2
)

# 提取图结构
graph = cg.G
print("\n✅ PC 算法完成")
print(f"节点数: {graph.num_vars}")

# 可视化（使用 networkx）
def visualize_causal_graph(graph, var_names, title="Learned Causal Graph"):
    """
    可视化因果图
    """
    G = nx.DiGraph()
    G.add_nodes_from(var_names)
    
    # 提取边
    edges = graph.graph
    directed_edges = []
    undirected_edges = []
    
    for i in range(len(var_names)):
        for j in range(i + 1, len(var_names)):
            edge = edges[i, j]
            if edge == -1 and edges[j, i] == 1:
                # i -> j
                directed_edges.append((var_names[i], var_names[j]))
                G.add_edge(var_names[i], var_names[j])
            elif edge == 1 and edges[j, i] == -1:
                # i <- j
                directed_edges.append((var_names[j], var_names[i]))
                G.add_edge(var_names[j], var_names[i])
            elif edge == -1 and edges[j, i] == -1:
                # i - j (undirected)
                undirected_edges.append((var_names[i], var_names[j]))
    
    # 使用 Plotly 绘制
    pos = nx.spring_layout(G, k=2, iterations=50)
    
    # 边
    edge_trace = []
    for edge in directed_edges:
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=2, color='#2D9CDB'),
                hoverinfo='none',
                showlegend=False
            )
        )
        # 箭头
        edge_trace.append(
            go.Scatter(
                x=[x1],
                y=[y1],
                mode='markers',
                marker=dict(
                    size=15,
                    color='#2D9CDB',
                    symbol='arrow',
                    angle=np.arctan2(y1 - y0, x1 - x0) * 180 / np.pi
                ),
                hoverinfo='none',
                showlegend=False
            )
        )
    
    for edge in undirected_edges:
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                mode='lines',
                line=dict(width=2, color='#95A5A6', dash='dash'),
                hoverinfo='none',
                showlegend=False
            )
        )
    
    # 节点
    node_x = [pos[node][0] for node in var_names]
    node_y = [pos[node][1] for node in var_names]
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        text=var_names,
        textposition='top center',
        marker=dict(
            size=30,
            color='#27AE60',
            line=dict(width=2, color='white')
        ),
        hoverinfo='text',
        showlegend=False
    )
    
    # 组合
    fig = go.Figure(data=edge_trace + [node_trace])
    fig.update_layout(
        title=title,
        showlegend=False,
        hovermode='closest',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=600
    )
    
    return fig

fig = visualize_causal_graph(graph, var_names, "PC 算法学到的因果图")
fig.show()

print("\n图例:")
print("🔵 实线箭头: 确定的因果方向")
print("⚪ 虚线: 无法确定方向（马尔可夫等价）")

## 💻 TODO 1: 理解条件独立性检验

实现一个函数，检验给定条件集下的独立性，并解释结果。

In [ ]:
# 💻 练习 1: 实现条件独立性检验
# 提示：
# 1. 边缘独立性：使用 pearsonr(data[X], data[Y])
# 2. 条件独立性：计算偏相关
#    - 用 Z 预测 X，得到残差 res_X
#    - 用 Z 预测 Y，得到残差 res_Y
#    - 计算 res_X 和 res_Y 的相关性

def analyze_conditional_independence(data, X, Y, Z_list=None):
    """
    分析 X 和 Y 在不同条件集下的独立性
    
    参数:
        data: DataFrame
        X, Y: 变量名
        Z_list: 条件变量列表的列表 (例如 [[], ['A'], ['A', 'B']])
    
    返回:
        结果 DataFrame
    """
    if Z_list is None:
        Z_list = [[]]
    
    results = []
    
    # TODO: 实现独立性检验
    # 对每个条件集 Z in Z_list:
    #   1. 如果 Z 为空，计算边缘相关
    #   2. 否则，计算偏相关（控制 Z 后的 X-Y 相关性）
    #   3. 记录相关系数、p值、是否独立
    
    pass  # 替换为你的实现

# 测试你的实现
# result = analyze_conditional_independence(
#     df,
#     'PurchaseFreq',
#     'BrowseTime',
#     Z_list=[[], ['MemberLevel'], ['MemberLevel', 'Churn']]
# )
# print(result)

<details>
<summary>📖 参考答案（点击展开）</summary>

```python
def analyze_conditional_independence(data, X, Y, Z_list=None):
    """
    分析 X 和 Y 在不同条件集下的独立性
    """
    if Z_list is None:
        Z_list = [[]]
    
    results = []
    
    from scipy.stats import pearsonr
    from sklearn.linear_model import LinearRegression
    
    for Z in Z_list:
        if len(Z) == 0:
            # 边缘相关
            corr, pval = pearsonr(data[X], data[Y])
        else:
            # 偏相关：控制 Z 后 X 和 Y 的相关性
            reg_X = LinearRegression().fit(data[Z], data[X])
            reg_Y = LinearRegression().fit(data[Z], data[Y])
            res_X = data[X] - reg_X.predict(data[Z])
            res_Y = data[Y] - reg_Y.predict(data[Z])
            corr, pval = pearsonr(res_X, res_Y)
        
        results.append({
            'Condition': str(Z) if Z else 'Marginal',
            'Correlation': corr,
            'P-value': pval,
            'Independent (α=0.05)': pval > 0.05
        })
    
    return pd.DataFrame(results)

# 测试
result = analyze_conditional_independence(
    df,
    'PurchaseFreq',
    'BrowseTime',
    Z_list=[[], ['MemberLevel'], ['MemberLevel', 'Churn']]
)
print(result)
```

**💡 观察**: 控制不同变量后，相关性如何变化？
</details>

---

# Part 3: 基于得分的方法 - GES 算法

## 3.1 核心思想

GES (Greedy Equivalence Search) 通过**优化评分函数**搜索最优因果图。

### 评分函数

**BIC (Bayesian Information Criterion)**:
$$
\text{BIC}(G) = \log L(D | G, \hat{\theta}) - \frac{k}{2} \log n
$$

- $L(D | G, \hat{\theta})$: 似然函数
- $k$: 参数个数
- $n$: 样本量

**目标**: $\arg\max_G \text{BIC}(G)$

### 算法流程

**Forward Phase (加边)**:
1. 从空图开始
2. 贪心地添加边，直到 BIC 不再增加

**Backward Phase (删边)**:
1. 从当前图开始
2. 贪心地删除边，直到 BIC 不再增加

### 与 PC 的对比

| 特性 | PC 算法 | GES 算法 |
|------|---------|----------|
| 方法 | 约束优化 | 评分优化 |
| 输入 | 独立性检验 | 评分函数 |
| 优点 | 直观、快速 | 全局最优 |
| 缺点 | 局部决策 | 计算昂贵 |

In [ ]:
# 运行 GES 算法
print("🔍 运行 GES 算法...")
Record = ges(
    data_array,
    score_func='local_score_BIC',
    maxP=None
)

ges_graph = Record['G']
print("\n✅ GES 算法完成")

# 可视化
fig = visualize_causal_graph(ges_graph, var_names, "GES 算法学到的因果图")
fig.show()

print("\n比较 PC 和 GES:")
print("PC: 基于条件独立性检验")
print("GES: 基于 BIC 评分优化")
print("\n两者可能产生不同的结果，尤其是在样本量较小时。")

## 💻 TODO 2: 比较不同算法

运行多个算法，比较它们的结果。

In [ ]:
# 💻 练习 2: 比较不同算法
# 提示：
# 1. 使用不同的 alpha 参数运行 PC 算法
# 2. 运行 GES 算法
# 3. 提取每个算法的边集合
# 4. 计算 Jaccard 相似度

def compare_causal_discovery_algorithms(data, var_names):
    """
    比较多个因果发现算法
    
    返回:
        各算法的边集合
    """
    # TODO: 实现算法比较
    # 1. 对不同 alpha 运行 PC 算法
    # 2. 运行 GES 算法
    # 3. 提取边集合
    pass

def extract_edges(graph, var_names):
    """从图对象提取边"""
    # TODO: 实现边提取逻辑
    # 检查邻接矩阵，找到所有有向边
    pass

# 测试你的实现
# results = compare_causal_discovery_algorithms(df, var_names)
# print(results)

<details>
<summary>📖 参考答案（点击展开）</summary>

```python
def compare_causal_discovery_algorithms(data, var_names):
    """比较多个因果发现算法"""
    data_array = data.values
    results = {}
    
    # 运行 PC 算法（不同超参数）
    for alpha in [0.01, 0.05, 0.1]:
        cg = pc(data_array, alpha=alpha, indep_test='fisherz', stable=True)
        edges = extract_edges(cg.G, var_names)
        results[f'PC (α={alpha})'] = edges
    
    # 运行 GES 算法
    Record = ges(data_array, score_func='local_score_BIC')
    edges = extract_edges(Record['G'], var_names)
    results['GES'] = edges
    
    return results

def extract_edges(graph, var_names):
    """从图对象提取边"""
    edges = []
    adj = graph.graph
    for i in range(len(var_names)):
        for j in range(len(var_names)):
            if adj[i, j] == -1 and adj[j, i] == 1:
                edges.append(f"{var_names[i]} → {var_names[j]}")
    return set(edges)

# 运行比较
results = compare_causal_discovery_algorithms(df, var_names)

print("算法比较结果:\n")
for algo, edges in results.items():
    print(f"{algo}:")
    for edge in sorted(edges):
        print(f"  {edge}")
    print()

# 计算算法间的一致性（Jaccard 相似度）
from itertools import combinations

print("算法一致性分析:")
print("\nJaccard 相似度矩阵:")
print("-" * 60)

algorithms = list(results.keys())
for algo1, algo2 in combinations(algorithms, 2):
    edges1 = results[algo1]
    edges2 = results[algo2]
    
    intersection = len(edges1 & edges2)
    union = len(edges1 | edges2)
    jaccard = intersection / union if union > 0 else 0
    
    print(f"{algo1:<15} vs {algo2:<15}: {jaccard:.3f}")
```

</details>

In [ ]:
# 辅助函数：提取因果图的有向边
# （供后续示范代码使用）
def extract_edges(graph, var_names):
    """从图对象提取边"""
    edges = []
    adj = graph.graph
    for i in range(len(var_names)):
        for j in range(len(var_names)):
            if adj[i, j] == -1 and adj[j, i] == 1:
                edges.append(f"{var_names[i]} → {var_names[j]}")
    return set(edges)

---

# Part 4: 函数因果模型

## 4.1 LiNGAM (Linear Non-Gaussian Acyclic Model)

### 核心思想

利用**非高斯性**识别因果方向。

**模型假设**:
$$
x_i = \sum_{j \in \text{Pa}(i)} b_{ij} x_j + e_i
$$

其中:
- $e_i$ 是非高斯噪声
- $e_i \perp\!\!\!\perp e_j, \forall i \neq j$

**关键洞察**:
- 如果 $X \rightarrow Y$: $Y = f(X) + e_Y$，则 $X \perp\!\!\!\perp e_Y$
- 如果 $Y \rightarrow X$: $X = g(Y) + e_X$，则 $Y \perp\!\!\!\perp e_X$
- 对于非高斯数据，两者**不能同时成立**！

### 算法

**DirectLiNGAM**:
1. 找到外生变量（残差最非高斯）
2. 回归消除其影响
3. 重复直到所有变量排序

In [ ]:
# 演示 LiNGAM 的思想
def demonstrate_nongaussian_identifiability():
    """
    演示非高斯性如何识别因果方向
    """
    n = 2000
    
    # Case 1: X -> Y (非高斯噪声)
    X = np.random.laplace(0, 1, n)  # Laplace 分布（非高斯）
    e_Y = np.random.laplace(0, 0.5, n)
    Y = 2 * X + e_Y
    
    # 测试两个方向
    from sklearn.linear_model import LinearRegression
    from scipy.stats import normaltest
    
    # X -> Y
    reg_xy = LinearRegression().fit(X.reshape(-1, 1), Y)
    residual_xy = Y - reg_xy.predict(X.reshape(-1, 1))
    _, pval_xy = normaltest(residual_xy)
    
    # Y -> X
    reg_yx = LinearRegression().fit(Y.reshape(-1, 1), X)
    residual_yx = X - reg_yx.predict(Y.reshape(-1, 1))
    _, pval_yx = normaltest(residual_yx)
    
    # 独立性检验
    from scipy.stats import pearsonr
    corr_x_res_xy, _ = pearsonr(X, residual_xy)
    corr_y_res_yx, _ = pearsonr(Y, residual_yx)
    
    print("非高斯性识别因果方向:\n")
    print(f"假设 X → Y:")
    print(f"  残差非高斯性 p-value: {pval_xy:.4f}")
    print(f"  X ⊥ residual: {abs(corr_x_res_xy):.4f}")
    print(f"\n假设 Y → X:")
    print(f"  残差非高斯性 p-value: {pval_yx:.4f}")
    print(f"  Y ⊥ residual: {abs(corr_y_res_yx):.4f}")
    print(f"\n✅ 结论: X → Y 更可能（残差更非高斯且更独立）")
    
    # 可视化
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=[
            "X 的分布（非高斯）",
            "Y 的分布",
            "假设 X → Y 的残差",
            "假设 Y → X 的残差"
        ]
    )
    
    fig.add_trace(go.Histogram(x=X, name='X', marker_color='#2D9CDB', nbinsx=50), row=1, col=1)
    fig.add_trace(go.Histogram(x=Y, name='Y', marker_color='#27AE60', nbinsx=50), row=1, col=2)
    fig.add_trace(go.Histogram(x=residual_xy, name='Residual X→Y', marker_color='#9B59B6', nbinsx=50), row=2, col=1)
    fig.add_trace(go.Histogram(x=residual_yx, name='Residual Y→X', marker_color='#E74C3C', nbinsx=50), row=2, col=2)
    
    fig.update_layout(height=600, title_text="LiNGAM 原理: 非高斯性识别因果方向", showlegend=False)
    fig.show()

demonstrate_nongaussian_identifiability()

## 4.2 应用 LiNGAM 到业务数据

注意：LiNGAM 需要**线性关系**和**非高斯噪声**。我们的数据部分满足这些假设。

In [ ]:
try:
    import lingam
except ImportError:
    print("安装 lingam...")
    import sys
    !{sys.executable} -m pip install lingam -q
    import lingam

# 移除离散变量（LiNGAM 要求连续）
df_continuous = df[['PurchaseFreq', 'BrowseTime', 'CouponUsage']].copy()

# 运行 DirectLiNGAM
print("🔍 运行 DirectLiNGAM...")
model = lingam.DirectLiNGAM()
model.fit(df_continuous)

print("\n因果顺序:")
for i, var_idx in enumerate(model.causal_order_):
    print(f"{i+1}. {df_continuous.columns[var_idx]}")

print("\n邻接矩阵 (i -> j):")
adj_matrix = pd.DataFrame(
    model.adjacency_matrix_,
    columns=df_continuous.columns,
    index=df_continuous.columns
)
print(adj_matrix.round(3))

# 可视化
fig = go.Figure(data=go.Heatmap(
    z=adj_matrix.values,
    x=adj_matrix.columns,
    y=adj_matrix.index,
    colorscale='Blues',
    text=adj_matrix.values,
    texttemplate='%{text:.2f}',
    textfont={"size": 12}
))

fig.update_layout(
    title="LiNGAM 因果强度矩阵 (行 → 列)",
    xaxis_title="To",
    yaxis_title="From",
    height=400
)
fig.show()

print("\n💡 解读:")
print("非零元素 (i, j) 表示 i → j 的因果效应强度")

---

# Part 5: 实践注意事项

## 5.1 样本量要求

因果发现需要**大量数据**，尤其是变量多时。

In [ ]:
# 演示样本量的影响
def evaluate_sample_size_effect(true_graph_edges, sample_sizes, n_trials=5):
    """
    评估样本量对因果发现准确性的影响
    """
    results = []
    
    for n in sample_sizes:
        accuracies = []
        
        for trial in range(n_trials):
            # 生成数据
            data = generate_ecommerce_data(n)
            data_array = data.values
            
            # 运行 PC
            cg = pc(data_array, alpha=0.05, indep_test='fisherz', stable=True)
            learned_edges = extract_edges(cg.G, list(data.columns))
            
            # 计算准确率
            tp = len(learned_edges & true_graph_edges)
            fp = len(learned_edges - true_graph_edges)
            fn = len(true_graph_edges - learned_edges)
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
            
            accuracies.append(f1)
        
        results.append({
            'Sample Size': n,
            'F1 Score': np.mean(accuracies),
            'Std': np.std(accuracies)
        })
    
    return pd.DataFrame(results)

# 真实因果边
true_edges = {
    'MemberLevel → PurchaseFreq',
    'MemberLevel → CouponUsage',
    'PurchaseFreq → BrowseTime',
    'Complaints → CouponUsage',
    'BrowseTime → Churn',
    'CouponUsage → Churn',
    'Complaints → Churn'
}

print("🔍 评估样本量影响...")
results_df = evaluate_sample_size_effect(
    true_edges,
    sample_sizes=[200, 500, 1000, 2000, 5000],
    n_trials=3
)

print(results_df)

# 可视化
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=results_df['Sample Size'],
    y=results_df['F1 Score'],
    mode='lines+markers',
    name='F1 Score',
    line=dict(color='#2D9CDB', width=3),
    marker=dict(size=10),
    error_y=dict(
        type='data',
        array=results_df['Std'],
        visible=True
    )
))

fig.update_layout(
    title="因果发现准确率 vs 样本量",
    xaxis_title="Sample Size",
    yaxis_title="F1 Score",
    yaxis=dict(range=[0, 1]),
    height=400
)
fig.show()

print("\n📊 关键发现:")
print("1. 样本量越大，因果发现越准确")
print("2. 至少需要 1000+ 样本才能获得稳定结果")
print("3. 小样本 (<500) 可能产生大量错误")

## 5.2 假设验证

因果发现算法都有**强假设**，需要验证。

In [ ]:
# 💻 练习 3: 解释因果发现结果
# 提示：
# 1. 从邻接矩阵中找到目标变量的直接原因
# 2. 使用 networkx 查找间接路径
# 3. 找到根本原因（无入边的节点）
# 4. 生成业务建议

def interpret_causal_graph(graph, var_names, target='Churn'):
    """
    解释因果图，生成业务建议
    
    参数:
        graph: 学到的因果图
        var_names: 变量名列表
        target: 目标变量
    
    返回:
        业务建议字典
    """
    # TODO: 实现因果图解释
    # 1. 找到 target 的直接原因（邻接矩阵中指向 target 的边）
    # 2. 使用 networkx 找到间接原因（有路径但不直接相连）
    # 3. 找到根本原因（外生变量，无入边）
    # 4. 生成业务建议
    
    pass

# 测试你的实现
# insights = interpret_causal_graph(graph, var_names, target='Churn')

<details>
<summary>📖 参考答案（点击展开）</summary>

```python
def interpret_causal_graph(graph, var_names, target='Churn'):
    """解释因果图，生成业务建议"""
    target_idx = var_names.index(target)
    adj = graph.graph
    
    # 找到 target 的直接原因
    direct_causes = []
    for i in range(len(var_names)):
        if adj[i, target_idx] == -1 and adj[target_idx, i] == 1:
            direct_causes.append(var_names[i])
    
    print(f"📊 {target} 的因果分析\n" + "="*50)
    print(f"\n直接原因 (可干预):")
    for cause in direct_causes:
        print(f"  ✅ {cause} → {target}")
    
    # 找到间接原因（通过中介）
    G = nx.DiGraph()
    for i in range(len(var_names)):
        for j in range(len(var_names)):
            if adj[i, j] == -1 and adj[j, i] == 1:
                G.add_edge(var_names[i], var_names[j])
    
    indirect_causes = []
    for var in var_names:
        if var != target and var not in direct_causes:
            if nx.has_path(G, var, target):
                indirect_causes.append(var)
    
    print(f"\n间接原因 (需谨慎):")
    for cause in indirect_causes:
        try:
            path = nx.shortest_path(G, cause, target)
            print(f"  🔗 {' → '.join(path)}")
        except:
            pass
    
    # 找到根本原因（无入边）
    root_causes = []
    for i in range(len(var_names)):
        has_parent = False
        for j in range(len(var_names)):
            if adj[j, i] == -1 and adj[i, j] == 1:
                has_parent = True
                break
        if not has_parent and var_names[i] != target:
            root_causes.append(var_names[i])
    
    print(f"\n根本原因 (外生变量):")
    for cause in root_causes:
        print(f"  🎯 {cause} (最上游)")
    
    # 生成干预建议
    print(f"\n💡 业务建议:")
    print("1. 优先干预直接原因 (见效快):")
    for cause in direct_causes:
        print(f"   - 改善 {cause}")
    
    print("\n2. 长期优化根本原因 (治本):")
    for cause in root_causes:
        print(f"   - 提升 {cause}")
    
    return {
        'direct_causes': direct_causes,
        'indirect_causes': indirect_causes,
        'root_causes': root_causes
    }

# 使用 PC 算法的结果
insights = interpret_causal_graph(graph, var_names, target='Churn')
```

</details>

## 💻 TODO 3: 解释因果发现结果

将学到的因果图转化为业务洞察。

In [ ]:
# 完整实现因果图解释
def interpret_causal_graph(graph, var_names, target='Churn'):
    """
    解释因果图，生成业务建议
    
    参数:
        graph: 学到的因果图
        var_names: 变量名列表
        target: 目标变量
    
    返回:
        业务建议字典
    """
    target_idx = var_names.index(target)
    adj = graph.graph
    
    # 找到 target 的直接原因
    direct_causes = []
    for i in range(len(var_names)):
        if adj[i, target_idx] == -1 and adj[target_idx, i] == 1:
            direct_causes.append(var_names[i])
    
    print(f"📊 {target} 的因果分析\n" + "="*50)
    print(f"\n直接原因 (可干预):")
    for cause in direct_causes:
        print(f"  ✅ {cause} → {target}")
    
    # 找到间接原因（通过中介）
    # 使用 networkx 查找路径
    G = nx.DiGraph()
    for i in range(len(var_names)):
        for j in range(len(var_names)):
            if adj[i, j] == -1 and adj[j, i] == 1:
                G.add_edge(var_names[i], var_names[j])
    
    indirect_causes = []
    for i, var in enumerate(var_names):
        if var != target and var not in direct_causes:
            # 检查是否有路径到 target
            if nx.has_path(G, var, target):
                indirect_causes.append(var)
    
    print(f"\n间接原因 (需谨慎):")
    for cause in indirect_causes:
        # 找一条路径示例
        try:
            path = nx.shortest_path(G, cause, target)
            path_str = " → ".join(path)
            print(f"  🔗 {path_str}")
        except:
            pass
    
    # 找到根本原因（无入边）
    root_causes = []
    for i in range(len(var_names)):
        has_parent = False
        for j in range(len(var_names)):
            if adj[j, i] == -1 and adj[i, j] == 1:
                has_parent = True
                break
        if not has_parent and var_names[i] != target:
            root_causes.append(var_names[i])
    
    print(f"\n根本原因 (外生变量):")
    for cause in root_causes:
        print(f"  🎯 {cause} (最上游)")
    
    # 生成干预建议
    print(f"\n💡 业务建议:")
    print("1. 优先干预直接原因 (见效快):")
    for cause in direct_causes:
        print(f"   - 改善 {cause}")
    
    print("\n2. 长期优化根本原因 (治本):")
    for cause in root_causes:
        if cause != target:
            print(f"   - 提升 {cause}")
    
    print("\n3. 注意中介效应:")
    if indirect_causes:
        print(f"   - 干预 {indirect_causes[0]} 等变量可能通过中介起作用")
    else:
        print("   - 当前未发现明显的中介路径")
    
    return {
        'direct_causes': direct_causes,
        'indirect_causes': indirect_causes,
        'root_causes': root_causes
    }

# 使用 PC 算法的结果
insights = interpret_causal_graph(graph, var_names, target='Churn')

## 5.3 因果发现的局限性

**关键限制**:

1. **马尔可夫等价类**
   - 某些边的方向无法唯一确定
   - 需要额外假设或领域知识

2. **假设敏感性**
   - 违反假设会导致错误结果
   - 难以验证所有假设

3. **计算复杂度**
   - 变量多时指数级增长
   - 需要大样本量

4. **隐变量**
   - 未观测混淆会产生虚假边
   - FCI 算法可部分处理

5. **时间信息**
   - 横截面数据难以确定方向
   - 时间序列数据更可靠

**最佳实践**:

```python
# 组合多种方法
results = {
    'PC': run_pc(data),
    'GES': run_ges(data),
    'LiNGAM': run_lingam(data),
}

# 取交集（保守）
consensus = set.intersection(*[set(r.edges) for r in results.values()])

# 结合领域知识
validated = filter_with_domain_knowledge(consensus)
```

---

# 练习与思考题

## 练习 1: 敏感性分析

改变 PC 算法的 $\alpha$ 参数，观察结果如何变化。

```python
for alpha in [0.001, 0.01, 0.05, 0.1, 0.2]:
    graph = run_pc(data, alpha=alpha)
    print(f"α={alpha}: {count_edges(graph)} edges")
```

**思考**: 
- $\alpha$ 越大，边越多还是越少？为什么？
- 如何选择合适的 $\alpha$？

## 练习 2: 对比真实结构

我们知道数据生成的真实因果图。计算学到的图与真实图的差异。

```python
true_edges = {...}  # 真实边集
learned_edges = {...}  # 学到的边集

# 计算 Precision, Recall, F1
```

## 练习 3: 处理隐变量

尝试使用 FCI 算法处理可能存在的隐变量。

```python
from causallearn.search.ConstraintBased.FCI import fci

graph_fci = fci(data, alpha=0.05)
# 比较 FCI 和 PC 的结果
```

## 思考题

1. **因果发现 vs 因果推断**：两者的区别和联系是什么？什么时候用哪个？

2. **时间序列**：如果有时间戳，因果发现会更容易吗？为什么？

3. **干预实验**：如果可以做 A/B 测试，还需要因果发现吗？

4. **业务应用**：在你的业务场景中，哪些问题适合用因果发现？哪些不适合？

5. **伦理问题**：自动学习因果关系可能带来什么风险？如何规避？

---

# 总结

## 关键要点

1. **因果发现的目标**
   - 从数据学习因果结构
   - 区分相关和因果

2. **主要方法**
   - 基于约束: PC, FCI (条件独立性)
   - 基于得分: GES (BIC 优化)
   - 函数因果模型: LiNGAM (非高斯性)

3. **实践建议**
   - 需要大样本 (1000+)
   - 验证假设
   - 结合领域知识
   - 使用多种方法交叉验证

4. **局限性**
   - 马尔可夫等价
   - 隐变量问题
   - 强假设依赖

## 进一步学习

- **书籍**: 
  - *Causation, Prediction, and Search* (Spirtes et al.)
  - *Elements of Causal Inference* (Peters et al.)

- **工具**: 
  - causal-learn (Python)
  - pcalg (R)
  - Tetrad (GUI)

- **前沿**:
  - 深度学习 + 因果发现
  - 非线性因果发现
  - 因果表征学习

## 下一步

学习**因果推断与机器学习的结合**：
- Causal ML
- Meta-learners
- Double ML

---

**恭喜你完成因果发现的学习！🎉**

你已经掌握了从数据中自动学习因果结构的方法。结合前面学到的因果推断技术，你现在可以：
1. 发现因果关系 (Causal Discovery)
2. 估计因果效应 (Causal Inference)
3. 预测干预结果 (Causal Prediction)

继续探索因果科学的奇妙世界吧！